# Diffusion data processing using FSL

![flowchart](dwi_flowchart.png)

### Commands for pre-pre-processing
- convert from dicoms to nifti/bval/bvec
- extract B0 volume of the nifti file (assuming it's the first volume)
- skullstrip the B0 volume

In [ ]:
%%bash
cd  DTIprocessing/Subj1
dcm2nii DICOM_folder
mv DICOM_folder/*.nii.gz Subj1.nii.gz
mv DICOM_folder/*.bvec bvec.bvec
mv DICOM_folder/*.bval bval.bval
fslroi Subj1.nii.gz b0.nii.gz 0 1
bet  b0.nii.gz nodif_brain –f 0.4 –g 0 –m

### Commands for preprocessing 

#### Case 1
If you have only one dti data per subject AND no field mapping data collected, then the only preprocessing that can be done is eddy current correction

In [2]:
%%bash
eddy_correct Subj1.nii.gz  data 0

Input does not exist or is not in a supported format


If you are using FSL 5.0.11

In [ ]:
%%bash
eddy --imain=Sub1.nii.gz \
--mask=nodif_brain_mask.nii.gz \
--acqp=acqparams.txt \
--index=index.txt \
--bvecs=bvec.bvec \
--bvals=bval.bval \
--topup=my_topup_results \
--niter=8 --fwhm=10,8,4,2,0,0,0,0 \
--repol \
--out=data \
--mporder=6 \
--slspec=my_slspec.txt \
--s2v_niter=5 \
--s2v_lambda=1 \
--s2v_interp=trilinear

where acqparams.txt is a textfile with entries 

```
0 1 0 ro
```
in single row. ro stands for read out time, a parameter that you need to find out from the acquistion protocol. 
Create this file! Similarly, index.txt is a text file with as many 1's as the # volumes in your Sub1.nii.gz. The 1's can be entered as columns or rows.

After this, you are ready for fitting tensor model:

In [ ]:
%%bash
dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

#### Case 2
If you have two sets of data collected with phase encoding polarity switched, then preprocessing for geometric distortion correction can be done prior to eddy current correction. For older FSL versions, 


In [ ]:
%%bash
cd DTIprocessing/Subj1
dcm2nii DICOM_folder1
dcm2nii DICOM_folder2

mv DICOM_folder1/*.nii.gz Subj1_A2P.nii.gz
mv DICOM_folder2/*.nii.gz Subj1_P2A.nii.gz

fslroi Subj1_A2P.nii.gz b0_A2P.nii.gz 0 1 # the numbers are the dimensions of the data
fslroi Subj1_P2A.nii.gz b0_P2A.nii.gz 0 1

fslmerge -t A2P_P2A_b0 b0_A2P b0_P2A

topup --imain=A2P_P2A_b0 \
--datain=acqparams.txt \
--config=b02b0.cnf \
--out=topup_results \
--iout=my_hifi_b0 \
--fout=my_hifi_fout

applytopup --imain=Subj1_A2P,Subj1_P2A \
--inindex=1,2 \
--datatin=acqparams.txt \
--topup=topup_results \
--out=my_hifi_images

eddy_correct my_hifi_images.nii.gz data 0

fslmaths my_hifi_b0 -Tmean hifi_nodif

bet hifi_nodif.nii.gz nodif_brain –f 0.4 –g 0 –m

dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

For new FSL version:

In [ ]:
%%bash
fslmerge -t A2P_P2A Subj1_A2P Subj1_PA_P2A

eddy --imain=A2P_P2A.nii.gz \
--mask=nodif_brain_mask.nii.gz \
--acqp=acqparams.txt \
--index=index.txt \
--bvecs=bvec.bvec \
--bvals=bval.bval \
--topup=topup_results \
--niter=8 \
--fwhm=10,8,4,2,0,0,0,0 \
--repol \
--out=data \
--mporder=6 \
--slspec=my_slspec.txt \
--s2v_niter=5 \
--s2v_lambda=1 \
--s2v_interp=trilinear

dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

where acqparams.txt is a textfile with entries
```
0 -1 0 ro
0 1 0 ro 
```
in two rows. Similarly, index.txt is a text file with as many 1s as the # volumes in your Sub1_A2P.nii.gz and as many 2s as the # volumes in Sub1_P2A.nii.gz 

Check the output of dtifit and make sure the diffusion directions are estimated correctly!!At this point, you may decide to work with the scalar maps and do a statistical processing of that. To bring all subjects to a common space, you may also run a registration using FLIRT or FNIRT.


In [ ]:
%%bash
flirt -in nodif_brain.nii.gz \
-ref struct_betted.nii.gz \
-omat xfms/diff2str.mat \
-searchrx -90 90 \
-searchry -90 90 \
-searchrz -90 90 \
-dof 6 -cost corratio

convert_xfm -omat xfms/str2diff.mat -inverse xfms/diff2str.mat

flirt -in struct_betted.nii.gz \
-ref MNI152_T1_1mm_brain.nii.gz \
-omat xfms/str2standard.mat \
-searchrx -90 90 \
-searchry -90 90 \
-searchrz -90 90 \
-dof 12 -cost corratio

convert_xfm -omat xfms/standard2str.mat -inverse xfms/str2standard.mat

convert_xfm -omat xfms/diff2standard.mat -concat xfms/str2standard.mat xfms/diff2str.mat

convert_xfm -omat xfms/standard2diff.mat -inverse xfms/diff2standard.mat


Using FNIRT:

In [ ]:
fnirt --in=/struct.nii.gz \
--aff=xfms/str2standard.mat \
--cout=xfms/str2standard_warp \
--config=T1_2_MNI152_2mm

For whole brain voxel-wise statistical analysis, you may use feat’s GLM, or randomize or say take it to SPM, AFNI, R and perform analysis there. Alternatively you can do a tbss analysis.

 For ROI-based analysis, you can extract mean FA values from, say roixyz.nii.gz, as:

In [ ]:
fslmerge -t allFA.nii.gz Subj1_FA.nii.gz Subj2_FA.nii.gz.......
fslstats -k roixyz.nii.gz allFA.nii.gz -M

### Fiber Tracking Using FSL

After dtifit has been successfully run and the fiber directions checked for accuracy, run:

In [ ]:
%%bash
bedpostx  Subj1 –n 3

(three fiber per voxel)
can also work on multi-shell data.

	Come back after 24 hours and we are ready for fiber tracking!!

#### probtrackx2
Example fiber tracking from an MNI seed mask:

In [ ]:
%%bash
probtrackx2 \
-x juelich_prob_WMOpticradiationL_mask1.nii.gz  \
-l --onewaycondition \
-c 0.2 -S 2000 --steplength=0.5 -P 5000 \
--fibthresh=0.01 --distthresh=0.0 \
--sampvox=0.0 \
--xfm=subj1_model2_3fibres.bedpostX/xfms/standard2diff_warp.nii.gz \
--invxfm=subj1_model2_3fibres.bedpostX/xfms/diff2standard_warp.nii.gz \
--forcedir --opd \
-s subj1_model2_3fibres.bedpostX/merged \
-m subj1_model2_3fibres.bedpostX/nodif_brain_mask \
--dir=subj1_model2_3fibres.bedpostX/mm_mni